# Using SDBN Click Model To Overcome Position Bias

This section we use the _Simplified Dynamic Bayesian Network_ (SDBN) to overcome the position bias that we saw with direct Click-Through-Rate. We consider the SDBN judgments and how they compare to just the click through rate.

In [1]:
import sys
sys.path.append('..')
from aips import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ltr.sdbn_functions import *
from session_gen import SessionGenerator

# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
all_sessions()

,sess_id,query,rank,doc_id,clicked
0,2,ipad,0.0,92636260712,False
1,2,ipad,1.0,635753493559,True
2,2,ipad,2.0,885909393404,False
3,2,ipad,3.0,843404073153,False
4,2,ipad,4.0,885909457595,False
...,...,...,...,...,...
149995,60001,bluray,25.0,23942973416,False
149996,60001,bluray,26.0,25192107191,False
149997,60001,bluray,27.0,27242809710,False
149998,60001,bluray,28.0,600603132872,False


In [3]:
sessions = all_sessions()
products = fetch_products(doc_ids=sessions['doc_id'].unique())
products

,image,upc,name,name_txt_en_split,manufacturer,shortDescription,longDescription,id,_version_,promotion_b
0,"<img height=""100"" src=""../data/retrotech/image...",885909457588,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE;,"9.7"" widescreen display; 802.11a/b/g/n Wi-Fi; ...",The all-new thinner and lighter design makes i...,508934c3-feb0-46e5-9881-b877cbab2b3c,1791713686203662367,NaN
1,"<img height=""100"" src=""../data/retrotech/image...",36725729872,"Samsung - 15.6"" Laptop - 6GB Memory - 640GB Ha...","Samsung - 15.6"" Laptop - 6GB Memory - 640GB Ha...",Samsung,Windows 7 Home Premium 64-bitTechnical details...,This laptop features a 640GB hard drive for ab...,6604ffb6-4c24-4ad3-961a-26b60daaf201,1791713686223585298,NaN
2,"<img height=""100"" src=""../data/retrotech/image...",814916010240,Amazon - Kindle DX - Graphite,Amazon - Kindle DX - Graphite,Amazon,"9.7"" display with E-Ink technology; supports P...","Store up to 3,500 eBooks on this Kindle digita...",358cdf8f-b22a-4369-adf7-4119ddc90d8c,1791713686410231821,NaN
3,"<img height=""100"" src=""../data/retrotech/image...",885909432257,Apple&#xAE; - MacBook&#xAE; Pro / Intel&#xAE; ...,Apple&#xAE; - MacBook&#xAE; Pro / Intel&#xAE; ...,Apple&#xAE;,ENERGY STAR QualifiedMac OS X 10.6 Snow Leopar...,This laptop features a built-in wireless LAN f...,64ef158f-6f2d-47f7-8c5a-4c591824843f,1791713686413377555,NaN
4,"<img height=""100"" src=""../data/retrotech/image...",883929139446,"Harry Potter and the Deathly Hallows, Part 1 -...","Harry Potter and the Deathly Hallows, Part 1 -...",\N,\N,\N,0b1af3d1-de48-46da-9f1e-1e91b98d66f3,1791713686428057637,NaN
...,...,...,...,...,...,...,...,...,...,...
307,"<img height=""100"" src=""../data/retrotech/image...",786936817232,Pirates Of The Caribbean: On Stranger Tides (2...,Pirates Of The Caribbean: On Stranger Tides (2...,\N,\N,\N,df2e2c77-cf3a-4660-9f9a-d568b81f34bc,1791713687126409237,NaN
308,"<img height=""100"" src=""../data/retrotech/image...",97361301747,Star Trek: Fan Collectives - DVD,Star Trek: Fan Collectives - DVD,\N,\N,\N,adb3c120-2f12-43b1-abb3-15f36c23e814,1791713686393454623,NaN
309,"<img height=""100"" src=""../data/retrotech/image...",25192073007,Blues Brothers (Rated) (Unrated) - Widescreen ...,Blues Brothers (Rated) (Unrated) - Widescreen ...,\N,\N,\N,213953c0-9861-428c-95f5-f4b69f5e49aa,1791713686394503174,NaN
310,"<img height=""100"" src=""../data/retrotech/image...",97361301747,Star Trek: Fan Collectives - Fullscreen AC3 Do...,Star Trek: Fan Collectives - Fullscreen AC3 Do...,\N,\N,\N,2cadd56e-6183-4958-bb38-d0954c20746d,1791713686741581825,NaN


# Listing 11.7

Click models overcome position bias by learning an examine probability on each ranking. SDBN tracks examines relative to the the last click. This code marks last click position per session so we can compute examine probabilities.

In [4]:
#%load -s calculate_examine_probability ../ltr/sdbn_functions.py
def calculate_examine_probability(sessions):
    last_click_per_session = sessions.groupby(["clicked", "sess_id"])["rank"].max()[True]
    sessions["last_click_rank"] = last_click_per_session
    sessions["examined"] = sessions["rank"] <= sessions["last_click_rank"]
    return sessions

In [5]:
sessions = get_sessions("dryer")
calculate_examine_probability(sessions)

,query,rank,doc_id,clicked,last_click_rank,examined
sess_id,,,,,,
2,dryer,0.0,12505451713,False,12.0,True
2,dryer,1.0,84691226727,True,12.0,True
2,dryer,2.0,883049066905,False,12.0,True
2,dryer,3.0,14381196320,False,12.0,True
2,dryer,4.0,74108056764,False,12.0,True
...,...,...,...,...,...,...
5001,dryer,15.0,36725561977,True,15.0,True
5001,dryer,16.0,36725578241,False,15.0,False
5001,dryer,17.0,12505527456,False,15.0,False


# Listing 11.8

Aggregate clicks and examine counts

In [6]:
#%load -s calculate_clicked_examined ../ltr/sdbn_functions.py
def calculate_clicked_examined(sessions):
    sessions = caclulate_examine_probability(sessions)
    return sessions[sessions["examined"]] \
        .groupby("doc_id")[["clicked", "examined"]].sum()

In [7]:
sessions = get_sessions("dryer")
calculate_clicked_examined(sessions)

,clicked,examined
doc_id,,
12505451713,355,2707
12505525766,268,974
12505527456,110,428
14381196320,217,1202
36172950027,97,971
36725561977,119,572
36725578241,130,477
48231011396,166,423
48231011402,213,818


# Listing 11.9

We compute a grade - a probability of relevance - by dividing the clicks by examines. This is the kind of dynamic 'click thru rate' of SDBN, that accounts for whether the result was actually seen by users, not just whether it was shown on the screen.

In [8]:
#%load -s calculate_grade ../ltr/sdbn_functions.py
def calculate_grade(sessions):
    sessions = calculate_clicked_examined(sessions)
    sessions["grade"] = sessions["clicked"] / sessions["examined"]
    return sessions.sort_values("grade", ascending=False)

In [15]:
sessions = get_sessions("dryer")
calculate_grade(sessions)

,clicked,examined,grade
doc_id,,,
856751002097,133,323,0.411765
48231011396,166,423,0.392435
84691226727,804,2541,0.316411
74108007469,208,708,0.293785
12505525766,268,974,0.275154
36725578241,130,477,0.272537
48231011402,213,818,0.260391
12505527456,110,428,0.257009
74108096487,235,1097,0.214221


In [13]:
render_judged(products, calculate_grade(sessions), grade_col="grade", label=f"SDBN judgments for q={query}")

,grade,image,upc,name,shortDescription
0,0.411765,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
1,0.392435,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
2,0.316411,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
3,0.293785,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
4,0.275154,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
5,0.272537,,36725578241,Samsung - 7.3 Cu. Ft. 7-Cycle Electric Dryer - White,Soft-touch dial controls; 7 preset drying cycles; 4 temperature settings; powdercoat drum; noise reduction package
6,0.260391,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
7,0.257009,,12505527456,"Smart Choice - 1/2"" Safety+PLUS Stainless-Steel Gas Dryer Connector","Safety+PLUS automatic shut-off valve; leak detection solution; pipe thread sealant; 60,500 BTU; CSA approved"
8,0.214221,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
9,0.208042,,36725561977,Samsung - 3.5 Cu. Ft. 6-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedSoft dial touch pad controls; 6 cycles; delay-start; child lock; Vibration Reduction Technology


# Figure 11.10 Source Code

In [14]:
sessions = get_sessions("transformers dark of the moon")
calculate_grade(sessions)

,clicked,examined,grade
doc_id,,,
97360810042,412,642,0.641745
400192926087,62,129,0.480620
97363560449,96,243,0.395062
97363532149,42,130,0.323077
93624956037,41,154,0.266234
47875842328,367,1531,0.239713
47875841420,217,960,0.226042
25192107191,176,1082,0.162662
786936817218,118,777,0.151866


Up next: [Dealing with Low Confidence Situations](3.SDBN-Confidence-Bias.ipynb)